# Network Performance Planner Installation guidance
This notebook outlines the setup and usage of the Network Performance Planner blueprint, which leverages an agentic AI workflow for dynamic telecom network optimization.

## Step 1: Clone the Repository

In [ ]:
!git clone https://github.com/bubbleran/Telco-Network-Configuration.git

## Step 2: Set Up Environment Variables
Before running any services, export your NVIDIA API key and prepare the local cache. If you do not have the NVIDIA API key, please follow these steps to create one:
1. Visit build.nvidia.com
2. Create an account
3. Click your account icon in the screen upper corner
4. Select API keys from the menu
5. Press Generate API key

Once you have a valid API key, edit the /Telco-Network-Configuration/config.yaml  and add the NVIDIA API KEY.
Also review the other configuration parameters for the blueprint.

In [ ]:
import yaml
import os
from os.path import expanduser

NGC_API_KEY=yaml.safe_load(open('Telco-Network-Configuration/config.yaml', 'r'))['nvidia_api_key']
os.environ["NGC_API_KEY"] = NGC_API_KEY

## Step 3: Run NIM Container (Optional - for Local NIM Hosting)

In [ ]:
os.environ["LOCAL_NIM_CACHE"]=expanduser("~/.cache/nim")
!mkdir -p {os.environ["LOCAL_NIM_CACHE"]}
!sudo chmod -R 777 {os.environ["LOCAL_NIM_CACHE"]}

!echo $NGC_API_KEY | docker login -u '$oauthtoken' --password-stdin nvcr.io 

!docker run -d --rm \
    --name llama3.1-70b-container \
    --gpus all \
    --shm-size=16GB \
    -e NGC_API_KEY={os.environ["NGC_API_KEY"]} \
    -v {os.environ["LOCAL_NIM_CACHE"]}:/opt/nim/.cache \
    -p 8000:8000 \
    nvcr.io/nim/meta/llama-3.1-70b-instruct:latest


It might take 5-10 minutes to setup the LLama-3.1-70b. Monitor the status of the NIM using the docker logs by running the command below. You can rerun the following command to check the status. Once it is ready, it should output `Uvicorn running on http://0.0.0.0:8000`

In [ ]:
!docker logs --tail 10 llama3.1-70b-container

To update the config.yaml file to use endpoints of this locally hosted NIMs instead of API catalog endpoints, set `NIM_mode: True`.

## Step 4: Set Up Python Environment

In [ ]:
!pip install virtualenv
!python3.10 -m virtualenv bp_env
!source bp_env/bin/activate && pip install -r Telco-Network-Configuration/requirements.txt

## Step 5: Start the Streamlit UI

In [ ]:
!source bp_env/bin/activate && cd Telco-Network-Configuration && streamlit run telco_planner_ui.py

## Step 6: Reset the Environment (Optional)

To stop the application, please stop the kernel of the above cell.

In [ ]:
!bash ./Telco-Network-Configuration/agentic_llm_workflow/reset.sh

In [ ]:
!docker stop llama3.1-70b-container

## Notes:
- This blueprint supports BubbleRAN simulation or USRP-based real-device environments.
- The agentic system includes Config, Validation, and Monitoring agents for continuous network optimization.